#  Deep Convolutional Generative Adversarial Network

Generative Adversarial Networks (GANs) are one of the most interesting ideas in computer science today. Two models are trained simultaneously by an adversarial process. A generator ("the artist") learns to create images that look real, while a discriminator ("the art critic") learns to tell real images apart from fakes.

>During training, the generator progressively becomes better at creating images that look real, while the discriminator becomes better at telling them apart. The process reaches equilibrium when the discriminator can no longer distinguish real images from fakes

In [116]:
import math, random
import PIL

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.cluster.hierarchy as sch

from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix, accuracy_score, zero_one_loss
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import MDS, Isomap, TSNE, LocallyLinearEmbedding
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.cluster import KMeans, AgglomerativeClustering, AffinityPropagation, DBSCAN, OPTICS, SpectralClustering

from xgboost import XGBClassifier, XGBRegressor, XGBRFClassifier, XGBRFRegressor

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# try:
#   from apyori import apriori
# except:
#   !pip install apyori
# finally:
#   from apyori import apriori
# try:
#   from umap import UMAP
# except:
#   !pip install umap-learn
# finally:
#   from umap import UMAP

# Make NumPy printouts easier to read.
np.set_printoptions(precision=2, suppress=True)

In [129]:
(train_images, train_labels), (X_test, y_test) = datasets.mnist.load_data()

In [131]:
train_images.shape

(60000, 28, 28)

In [119]:
# train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
# train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]

In [132]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
INPUT_SHAPE = 100

In [133]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset

<BatchDataset shapes: (256, None, 28, 28), types: tf.uint8>

## Generator

In [134]:
def make_generator_model():
    # generator = models.Sequential()
    # generator.add(layers.Dense(100, activation="relu", input_shape=[INPUT_SHAPE]))
    # generator.add(layers.Dense(150,activation='relu'))
    # generator.add(layers.Dense(784, activation="sigmoid")) # 28*28 = 784
    # generator.add(layers.Reshape([28,28]))
    
    generator = models.Sequential()
    generator.add(layers.Dense(7 * 7 * 128, activation="relu", input_shape=[codings_size]))
    generator.add(layers.Reshape([7, 7, 128]))
    generator.add(layers.BatchNormalization())
    generator.add(layers.Conv2DTranspose(64, kernel_size=5, strides=2, padding="same", activation="relu"))
    generator.add(layers.BatchNormalization())
    generator.add(layers.Conv2DTranspose(1, kernel_size=5, strides=2, padding="same", activation="tanh"))

    return generator

## Discriminator

In [135]:
def make_discriminator_model():
    # discriminator = models.Sequential()
    # discriminator.add(layers.Flatten(input_shape=[28,28]))
    # discriminator.add(layers.Dense(150,activation='relu'))
    # discriminator.add(layers.Dense(100,activation='relu'))
    # discriminator.add(layers.Dense(1,activation="sigmoid"))
    
    discriminator = models.Sequential()
    discriminator.add(layers.Conv2D(64, kernel_size=5, strides=2, padding="same", activation=LeakyReLU(0.3), input_shape=[28, 28, 1]))
    discriminator.add(layers.Dropout(0.5))
    discriminator.add(layers.Conv2D(128, kernel_size=5, strides=2, padding="same", activation=LeakyReLU(0.3)))
    discriminator.add(layers.Dropout(0.5))
    discriminator.add(layers.Flatten())
    discriminator.add(layers.Dense(1, activation="sigmoid"))

    return discriminator

In [136]:
# Instance creation
generator = make_generator_model()
discriminator = make_discriminator_model()

In [140]:
discriminator.compile(loss="binary_crossentropy", optimizer="adam")
discriminator.trainable = False

In [ ]:
GAN = models.Sequential([generator, discriminator])
GAN.compile(loss="binary_crossentropy", optimizer="adam")
GAN.summary()

## Train & Test

In [142]:
generator, discriminator = GAN.layers

for epoch in range(2):
    for X_batch in train_dataset:
        noise = tf.random.normal(shape=[BATCH_SIZE, INPUT_SHAPE])
        generated_image = generator(noise)
        fake_vs_real = tf.concat([generated_image, tf.dtypes.cast(X_batch,tf.float32)], axis=0)
        y = tf.constant([[0.]] * BATCH_SIZE + [[1.]] * BATCH_SIZE)
        discriminator.trainable = True
        discriminator.train_on_batch(fake_vs_real, y)

        noise = tf.random.normal(shape=[BATCH_SIZE, INPUT_SHAPE])
        y = tf.constant([[1.]] * BATCH_SIZE)
        discriminator.trainable = False
        GAN.train_on_batch(noise, y)

print("TRAINING COMPLETE")

TRAINING COMPLETE


In [143]:
# create a noise image
noise = tf.random.normal(shape=[BATCH_SIZE, INPUT_SHAPE])
# plt.imshow(noise)

# create image using noise
generated_image = generator(noise)
print(generated_image.shape)
# plt.imshow(generated_image[0], cmap='gray')

(256, 28, 28)


In [145]:
# for image in generated_image:
#     plt.imshow(image.numpy().reshape(28,28))
#     plt.show()